In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
tf.config.list_physical_devices('GPU')

2021-10-24 12:38:37.742418: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 12:38:37.746919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 12:38:37.747154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [5]:
building1_filenames = set(glob.glob("datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("datasets/building1/train/*.feather"))
test1_filenames = building1_filenames - training_filenames

In [6]:
import random
seed=10
random.seed(seed)
np.random.seed(seed)

In [7]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)

#training_data.head()

orientation_acc = training_data[["iphoneAccX", "iphoneAccY", "iphoneAccZ"]]
orientation_gyro = training_data[[ "iphoneGyroX", "iphoneGyroY", "iphoneGyroZ"]]
orientation_mag = training_data[[ "iphoneMagX", "iphoneMagY", "iphoneMagZ"]] 

#orientation_data.head()

In [8]:
orientation_target = training_data[["orientX", "orientY", "orientZ", "orientW"]] 
orientation_target.head()

,orientX,orientY,orientZ,orientW
0,0.006522,0.002056,-0.000054,0.999977
1,0.006549,0.002054,-0.000135,0.999976
2,0.006513,0.002045,-0.000189,0.999977
3,0.006550,0.002060,-0.000221,0.999976
4,0.006574,0.002041,-0.000326,0.999976


In [9]:
acc_np = orientation_acc.to_numpy()
gyro_np = orientation_gyro.to_numpy()
mag_np = orientation_mag.to_numpy()

In [10]:
orient_np = orientation_target.to_numpy()

In [11]:
def generate_training_samples(acc_, gyro_, mag_, orient_, batch_size = 64):    
    while True:
        xa_batch = np.zeros((batch_size,100,3))
        xg_batch = np.zeros((batch_size,100,3))
        xm_batch = np.zeros((batch_size,100,3))
        y_theta_batch = np.zeros((batch_size,4))
        y_sigma_batch = np.finfo(np.float32).eps* np.ones((batch_size,6)) 
        j = 0
        for i in range(len(orient_)):
            xa_batch[j,:,:] = acc_[i, :]
            xg_batch[j,:,:] = gyro_[i,:]
            xm_batch[j,:,:] = mag_[i,:]
            y_theta_batch[j,:] = orient_[i,:]
            j += 1
            if j >= batch_size:
                j = 0              
                yield([xa_batch,xg_batch, xm_batch],[y_theta_batch, y_sigma_batch])
            #---------------------------------------------------------------------------------------

In [12]:
checkpoints_path = os.path.join(os.getcwd(),'datasets/CheckPoints_Model_OrientNet')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_vloss_{val_loss:.4f}.hdf5')
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [13]:
orient_learning_rate = 0.0005 # convergence  within  20  epochs \ref{IDOL}

In [14]:
# Loss Function
class OreintLoss(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # custom loss L_orient = 0.5*(q [-] \hat{q})^T *Sig^{-1}* 
    W = 25
    ws = tf.constant([W,W,1,1,1,1],dtype=tf.float32)
    return tf.sqrt(tf.reduce_mean(ws*(y_pred - y_true)**2, axis=-1))

class MyLossP(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # y_pred = tf.convert_to_tensor_v2(y_pred)
    # y_true = tf.cast(y_true, y_pred.dtype)
    W = 25
    ws = tf.constant([W,W,1],dtype=tf.float32)
    return tf.sqrt(tf.reduce_mean(ws*(y_pred - y_true)**2, axis=-1))

class MyLossA(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # y_pred = tf.convert_to_tensor_v2(y_pred)
    # y_true = tf.cast(y_true, y_pred.dtype)
    
    return tf.sqrt(tf.reduce_mean((y_pred - y_true)**2, axis=-1))



In [15]:
class OreintNet():
    def build_model(self):
        # Building Network
        #1. Define inputs
        Acc_input = Input(shape=(100,3), batch_size=64 , name = 'Acc_input')    
        Gyro_input = Input(shape=(100,3),batch_size=64   , name = 'Gyro_input')
        Mag_input = Input(shape=(100,3), batch_size=64   , name = 'Mag_input')
        
        MergedLayer = Concatenate()([Acc_input , Gyro_input, Mag_input])
        #
        ##MergedLayer = Input(shape=(9) , name = 'imu_input')
        #2. LSTM
        LSTM1 = (LSTM(100, return_sequences=True))(MergedLayer) # , return_sequences=True
        LSTM2 = (LSTM(100, return_sequences=False))(LSTM1)
        print("Output shape of the LSTM layer's default output : ", LSTM2.shape)
        
        #3. Fully-Connected (Sigma)
        Dense1 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2)
        Dense2 = Dense(units=32, activation='tanh')(Dense1)
        theta_output = Dense(units=4, name='theta_out')(Dense2) # Sigma Outputs
        
        #4. Fully-Connected (Theta)
        Dense3 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2) # input_shape=(
        Dense4 = Dense(units=32, activation='tanh')(Dense3)
        sigma_output = Dense(units=6, name='sig_out')(Dense4) # Theta Outputs  activation='linear'

        # specifies the weight per loss
        losses = {
        "theta_out": MyLossP(),
        "sig_out": MyLossA(),
        }
        lossWeights = {"pos_out": 1.0, "att_out": 1.0}

        
        #5. Define and compile The model
        Network = Model([Acc_input,Gyro_input, Mag_input],  [theta_output , sigma_output])  #  [Acc_input,Gyro_input, Mag_input]
        Network.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate) ,metrics=['mape']) #loss=OreintLoss()
        return Network
        

In [16]:
#class OreintNet(Model):
    

In [17]:
network_builder = OreintNet()
orientation_network = network_builder.build_model()

2021-10-24 12:38:38.722630: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-24 12:38:38.723390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 12:38:38.723666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 12:38:38.723859: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Output shape of the LSTM layer's default output :  (64, 100)


In [18]:
orientation_network.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Acc_input (InputLayer)          [(64, 100, 3)]       0                                            
__________________________________________________________________________________________________
Gyro_input (InputLayer)         [(64, 100, 3)]       0                                            
__________________________________________________________________________________________________
Mag_input (InputLayer)          [(64, 100, 3)]       0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (64, 100, 9)         0           Acc_input[0][0]                  
                                                                 Gyro_input[0][0]             

In [19]:
Nepochs = 20
history = orientation_network.fit(
    generate_training_samples(acc_np, gyro_np, mag_np, orient_np),
    epochs=Nepochs,
    initial_epoch=initial_epoch,
    batch_size=64,
    callbacks = [check_point_callback]
)

Epoch 1/20


2021-10-24 12:38:39.576123: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-24 12:38:41.388858: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


  19386/Unknown - 194s 10ms/step - loss: 0.1636 - theta_out_loss: 0.1636 - sig_out_loss: 2.4054e-05 - theta_out_mape: 929.2021 - sig_out_mape: 677817.5625

In [ ]:
pos_learning_rate = 0.001